# 第三课：Transformer架构

**课程时长**: 1小时  
**课程目标**: 
- 理解Transformer的核心组件
- 掌握注意力机制的原理
- 为学习大语言模型打下基础

**参考论文**: [Attention Is All You Need](https://arxiv.org/abs/1706.03762) (2017)

---

## 目录

1. Transformer的诞生背景
2. 整体架构概览
3. 核心组件详解
   - 3.1 Embedding (词嵌入)
   - 3.2 Positional Encoding (位置编码)
   - 3.3 Self-Attention (自注意力)
   - 3.4 Multi-Head Attention (多头注意力)
   - 3.5 Feed-Forward Network (前馈网络)
   - 3.6 Residual Connection (残差连接)
   - 3.7 Layer Normalization (层归一化)
   - 3.8 Masking (掩码)
4. Mixture of Experts (MoE)
5. Transformer的优势与局限
6. 总结与展望

In [ ]:
# 导入必要的库（用于可视化）
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

# 设置中文显示
rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
rcParams['axes.unicode_minus'] = False

# 设置绘图风格
sns.set_style('whitegrid')

print("Environment setup complete!")

## 1. Transformer的诞生背景

### 1.1 序列处理的历史

在Transformer出现之前，处理序列数据（如文本、时间序列）主要依靠:

#### RNN家族 (1980s-2010s)
```
RNN → LSTM → GRU
```

**优点**:
- 天然处理序列数据
- 理论上可以捕捉长距离依赖

**缺点**:
- ❌ **顺序处理**: 无法并行化，训练慢
- ❌ **梯度消失**: 难以捕捉长距离依赖
- ❌ **记忆瓶颈**: 所有信息压缩到固定大小的隐状态

### 1.2 Attention的引入 (2015)

**Bahdanau Attention** (用于机器翻译):
- 允许解码器关注输入序列的不同位置
- 解决了RNN的信息瓶颈问题
- 但仍然依赖RNN作为骨干

### 1.3 Transformer的革命 (2017)

**核心思想**: "Attention Is All You Need"
- **完全抛弃RNN/CNN**
- **纯粹基于注意力机制**
- **并行化训练**
- **更好地捕捉长距离依赖**

### 1.4 Transformer的影响

```
Transformer (2017)
    ↓
├─ NLP: BERT (2018), GPT系列, T5, ...
├─ Vision: ViT (2020), DETR, ...
├─ Speech: Whisper, ...
└─ Multi-modal: CLIP, Flamingo, GPT-4, ...
```

Transformer已成为**现代深度学习的统一架构**！

## 2. 整体架构概览

原始Transformer是一个**编码器-解码器**架构（用于机器翻译）:

```
┌─────────────────────────────────────────────────┐
│                 Transformer                     │
├────────────────────┬────────────────────────────┤
│    Encoder (编码器) │    Decoder (解码器)        │
│                    │                            │
│  [Nx编码器层]       │   [Nx解码器层]             │
│    ↓               │      ↓                     │
│  Multi-Head Attn   │   Masked Multi-Head Attn   │
│    ↓               │      ↓                     │
│  Feed Forward      │   Cross Attention          │
│    ↓               │      ↓                     │
│  (每层有残差+归一化) │   Feed Forward             │
│                    │      ↓                     │
│                    │   (每层有残差+归一化)        │
└────────────────────┴────────────────────────────┘
        ↑                          ↓
    输入序列                    输出序列
```

### 2.2 现代变体

| 模型类型 | 架构 | 代表模型 | 应用场景 |
|---------|------|---------|----------|
| **Encoder-only** | 仅使用编码器 | BERT, RoBERTa | 文本理解、分类 |
| **Decoder-only** | 仅使用解码器 | GPT系列, LLaMA, Qwen | 文本生成、对话 |
| **Encoder-Decoder** | 完整架构 | T5, BART | 翻译、摘要 |

**当前趋势**: Decoder-only架构（GPT范式）占据主导地位，因为:
- 更简洁
- 更易扩展
- 性能更强（在足够数据下）

### 2.3 一个编码器层的详细结构

```
输入 (X)
  ↓
┌─────────────────────────┐
│  Multi-Head Attention   │
└─────────────────────────┘
  ↓
Add & Norm (残差 + 层归一化)
  ↓
┌─────────────────────────┐
│   Feed Forward Network  │
└─────────────────────────┘
  ↓
Add & Norm (残差 + 层归一化)
  ↓
输出 (传递给下一层)
```

## 3. 核心组件详解

### 3.1 Embedding (词嵌入)

#### 什么是Embedding？

将**离散的词**映射到**连续的向量空间**。

```
"你好" → [0.2, -0.5, 0.8, ..., 0.3]  (d维向量)
"世界" → [0.1, 0.7, -0.2, ..., 0.5]
```

#### 为什么需要Embedding？

1. **语义相似的词在向量空间中也接近**
   - "国王" - "男人" + "女人" ≈ "女王"
   
2. **神经网络需要数值输入**
   - 不能直接处理文本

3. **降维**
   - 词汇表大小: 50,000+
   - 嵌入维度: 512-1024

#### Embedding的实现

本质上是一个**查找表 (Lookup Table)**:

$$
E \in \mathbb{R}^{V \times d}
$$

其中:
- $V$ 是词汇表大小
- $d$ 是嵌入维度
- 每一行是一个词的嵌入向量

#### 训练方式

1. **随机初始化** + 端到端训练
2. **预训练词向量**: Word2Vec, GloVe, FastText
3. **上下文相关**: 现代LLM中每个词的表示取决于上下文

In [ ]:
# 简单的Embedding示例（概念演示）
vocab_size = 10000  # 词汇表大小
embedding_dim = 512  # 嵌入维度

# 创建embedding矩阵（随机初始化）
embedding_matrix = np.random.randn(vocab_size, embedding_dim) * 0.01

# 词索引
word_ids = [123, 456, 789]  # "我 爱 中国"

# 查找对应的词向量
word_embeddings = embedding_matrix[word_ids]

print(f"Embedding matrix shape: {embedding_matrix.shape}")
print(f"Word sequence: {word_ids}")
print(f"Word embeddings shape: {word_embeddings.shape}")
print(f"\nFirst word embedding (first 10 dimensions):\n{word_embeddings[0][:10]}")

### 3.2 Positional Encoding (位置编码)

#### 问题：Transformer没有序列概念

与RNN不同，Transformer的Self-Attention是**排列不变的** (permutation invariant):

```
Attention("我 爱 你") = Attention("你 爱 我") = Attention("爱 你 我")
```

这显然不对！我们需要**注入位置信息**。

#### 解决方案：位置编码

将位置信息编码成向量,加到词嵌入上:

$$
\text{Input} = \text{Embedding} + \text{Positional Encoding}
$$

#### 正弦位置编码（原始Transformer）

对于位置 $pos$ 和维度 $i$:

$$
PE_{(pos, 2i)} = \sin\left(\frac{pos}{10000^{2i/d}}\right)
$$

$$
PE_{(pos, 2i+1)} = \cos\left(\frac{pos}{10000^{2i/d}}\right)
$$

#### 为什么使用正弦函数？

1. **确定性**: 无需学习参数
2. **外推性**: 可以处理训练时未见过的长度
3. **相对位置**: $PE_{pos+k}$ 可以表示为 $PE_{pos}$ 的线性函数
4. **不同频率**: 不同维度有不同的波长，编码不同粒度的位置信息

#### 现代变体

- **Learnable Position Embedding**: BERT使用可学习的位置嵌入
- **Relative Positional Encoding**: Transformer-XL, T5
- **Rotary Position Embedding (RoPE)**: LLaMA, Qwen使用
- **ALiBi**: 在attention score中直接加bias

In [ ]:
# 正弦位置编码的实现与可视化
def positional_encoding(max_len, d_model):
    """
    生成正弦位置编码
    
    参数:
    - max_len: 最大序列长度
    - d_model: 嵌入维度
    """
    pe = np.zeros((max_len, d_model))
    position = np.arange(0, max_len)[:, np.newaxis]
    div_term = np.exp(np.arange(0, d_model, 2) * -(np.log(10000.0) / d_model))
    
    pe[:, 0::2] = np.sin(position * div_term)
    pe[:, 1::2] = np.cos(position * div_term)
    
    return pe

# 生成位置编码
max_len = 100
d_model = 512
pe = positional_encoding(max_len, d_model)

# 可视化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# 热力图：位置编码矩阵
im = ax1.imshow(pe, cmap='RdBu', aspect='auto')
ax1.set_xlabel('Dimension')
ax1.set_ylabel('Position')
ax1.set_title('Positional Encoding Heatmap')
plt.colorbar(im, ax=ax1)

# 曲线图：不同位置在前几个维度的编码值
for i in range(0, 20, 4):  # Display dimensions 0, 4, 8, 12, 16
    ax2.plot(pe[:, i], label=f'Dim {i}')
ax2.set_xlabel('Position')
ax2.set_ylabel('Encoding Value')
ax2.set_title('Positional Encoding Curves for Different Dimensions')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Positional encoding shape: {pe.shape}")
print(f"\nPosition 0 encoding (first 10 dimensions):\n{pe[0, :10]}")
print(f"\nPosition 50 encoding (first 10 dimensions):\n{pe[50, :10]}")

### 3.3 Self-Attention (自注意力)

Self-Attention是Transformer的**核心创新**。

#### 核心思想

对于序列中的每个词，计算它与序列中**所有词（包括自己）**的关联度，然后加权求和。

```
例句: "The animal didn't cross the street because it was too tired."

当处理"it"时，Self-Attention会:
- 高度关注"animal" ✅
- 较少关注"street"
- 几乎不关注"the"
```

#### Query, Key, Value (Q, K, V)

这是Attention的三个核心概念,类比搜索引擎:

- **Query (查询)**: 我在找什么？
- **Key (键)**: 我是什么？
- **Value (值)**: 我的内容是什么？

对于输入 $X \in \mathbb{R}^{n \times d}$:

$$
Q = XW^Q, \quad K = XW^K, \quad V = XW^V
$$

其中 $W^Q, W^K, W^V \in \mathbb{R}^{d \times d_k}$ 是可学习的参数。

#### Scaled Dot-Product Attention

$$
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V
$$

**计算步骤**:

1. **计算相似度**: $QK^T \in \mathbb{R}^{n \times n}$ （点积）
2. **缩放**: 除以 $\sqrt{d_k}$ （防止梯度消失）
3. **归一化**: Softmax（得到注意力权重）
4. **加权求和**: 乘以 $V$

#### 为什么要缩放？

当 $d_k$ 很大时，点积的方差也很大，导致softmax进入饱和区，梯度很小。

$$
\text{Var}(QK^T) = d_k \quad \Rightarrow \quad \text{Var}\left(\frac{QK^T}{\sqrt{d_k}}\right) = 1
$$

In [ ]:
# Self-Attention的简化实现（概念演示）
def scaled_dot_product_attention(Q, K, V, mask=None):
    """
    缩放点积注意力
    
    参数:
    - Q: Query矩阵 (n, d_k)
    - K: Key矩阵 (n, d_k)
    - V: Value矩阵 (n, d_v)
    - mask: 掩码（可选）
    
    返回:
    - output: 注意力输出 (n, d_v)
    - attention_weights: 注意力权重 (n, n)
    """
    d_k = Q.shape[-1]
    
    # 1. 计算注意力分数 (相似度)
    scores = np.dot(Q, K.T) / np.sqrt(d_k)
    
    # 2. 应用掩码（如果有）
    if mask is not None:
        scores = scores + (mask * -1e9)
    
    # 3. Softmax归一化
    attention_weights = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
    attention_weights = attention_weights / np.sum(attention_weights, axis=-1, keepdims=True)
    
    # 4. 加权求和
    output = np.dot(attention_weights, V)
    
    return output, attention_weights

# 示例：3个词的序列
seq_len = 3
d_k = 4

# 随机生成Q, K, V（实际应用中是通过线性变换得到）
np.random.seed(42)
Q = np.random.randn(seq_len, d_k)
K = np.random.randn(seq_len, d_k)
V = np.random.randn(seq_len, d_k)

# 计算注意力
output, attn_weights = scaled_dot_product_attention(Q, K, V)

print("=== Self-Attention Example ===")
print(f"\nQuery shape: {Q.shape}")
print(f"Key shape: {K.shape}")
print(f"Value shape: {V.shape}")
print(f"\nAttention weights matrix:\n{attn_weights}")
print(f"\nRow sums (should all be 1.0):\n{attn_weights.sum(axis=1)}")
print(f"\nOutput shape: {output.shape}")

# 可视化注意力权重
fig, ax = plt.subplots(figsize=(6, 5))
im = ax.imshow(attn_weights, cmap='YlOrRd', vmin=0, vmax=1)
ax.set_xticks(range(seq_len))
ax.set_yticks(range(seq_len))
ax.set_xticklabels([f'Word{i+1}' for i in range(seq_len)])
ax.set_yticklabels([f'Word{i+1}' for i in range(seq_len)])
ax.set_xlabel('Key (Attended word)')
ax.set_ylabel('Query (Current word)')
ax.set_title('Attention Weights Heatmap')

# 在每个格子中显示数值
for i in range(seq_len):
    for j in range(seq_len):
        text = ax.text(j, i, f'{attn_weights[i, j]:.2f}',
                      ha="center", va="center", color="black", fontsize=12)

plt.colorbar(im, ax=ax)
plt.tight_layout()
plt.show()

### 3.4 Multi-Head Attention (多头注意力)

#### 动机：为什么需要多头？

单个注意力头可能只关注某一种模式,例如:
- 语法关系（主谓宾）
- 语义相关性
- 位置接近度

**多头注意力**允许模型同时关注**多种不同的模式**。

#### 核心思想

将Q, K, V分成 $h$ 个头,每个头独立计算attention,最后拼接:

$$
\text{MultiHead}(Q, K, V) = \text{Concat}(\text{head}_1, ..., \text{head}_h)W^O
$$

其中每个头:

$$
\text{head}_i = \text{Attention}(QW^Q_i, KW^K_i, VW^V_i)
$$

#### 参数详解

- $d_{\text{model}}$: 模型维度（如512）
- $h$: 头数（如8）
- $d_k = d_v = d_{\text{model}} / h$: 每个头的维度（如64）

#### 优势

1. **多样性**: 不同的头学习不同的注意力模式
2. **效率**: 总参数量与单头相当
3. **表达能力**: 增强模型的表示能力

#### 典型配置

| 模型 | $d_{\text{model}}$ | $h$ | $d_k$ |
|------|----------|-----|-------|
| BERT-base | 768 | 12 | 64 |
| GPT-3 | 12288 | 96 | 128 |
| LLaMA-7B | 4096 | 32 | 128 |
| Qwen-7B | 4096 | 32 | 128 |

In [ ]:
# 多头注意力的可视化示例
def visualize_multi_head_attention():
    """
    可视化多头注意力的概念
    """
    # 模拟一个句子 "I love natural language processing"
    words = ["I", "love", "natural", "language", "processing"]
    n = len(words)
    num_heads = 4
    
    # 为每个头生成不同的注意力模式
    np.random.seed(42)
    attention_patterns = []
    
    for i in range(num_heads):
        # 创建不同的注意力模式
        attn = np.random.rand(n, n)
        # 归一化
        attn = attn / attn.sum(axis=1, keepdims=True)
        attention_patterns.append(attn)
    
    # 可视化
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for idx, (ax, attn) in enumerate(zip(axes, attention_patterns)):
        im = ax.imshow(attn, cmap='YlOrRd', vmin=0, vmax=1)
        ax.set_xticks(range(n))
        ax.set_yticks(range(n))
        ax.set_xticklabels(words, rotation=45)
        ax.set_yticklabels(words)
        ax.set_title(f'Head {idx+1}')
        
        # 添加数值
        for i in range(n):
            for j in range(n):
                text = ax.text(j, i, f'{attn[i, j]:.2f}',
                              ha="center", va="center", 
                              color="black" if attn[i, j] < 0.5 else "white",
                              fontsize=8)
        
        plt.colorbar(im, ax=ax, fraction=0.046)
    
    plt.suptitle('Multi-Head Attention: Different Heads Focus on Different Patterns', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    print("Observations:")
    print("- Head 1 may focus on grammatical structure")
    print("- Head 2 may focus on semantic relationships")
    print("- Head 3 may focus on positional proximity")
    print("- Head 4 may focus on specific part-of-speech relationships")

visualize_multi_head_attention()

### 3.5 Feed-Forward Network (前馈网络)

每个Transformer层除了Multi-Head Attention,还有一个**Position-wise Feed-Forward Network**。

#### 结构

$$
\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2
$$

即两层全连接网络:

```
输入 (d_model)
  ↓
线性变换 + ReLU (d_ff = 4 * d_model)
  ↓
线性变换 (d_model)
  ↓
输出 (d_model)
```

#### 关键特点

1. **Position-wise**: 对每个位置独立应用相同的FFN
2. **扩张-压缩**: 中间层通常是4倍模型维度
3. **非线性**: 引入非线性变换能力

#### 作用

- **Attention负责信息聚合**: "看哪里"
- **FFN负责特征变换**: "做什么"

#### 现代变体

- **GELU激活**: GPT, BERT使用GELU替代ReLU
- **SwiGLU**: LLaMA, PaLM使用
  $$\text{SwiGLU}(x) = \text{Swish}(xW_1) \odot (xW_2)$$
- **Mixture of Experts (MoE)**: 用多个专家网络替代单一FFN（稍后详细讲解）

### 3.6 Residual Connection (残差连接)

#### 问题：深层网络的梯度消失

随着网络加深,梯度会逐层衰减,导致训练困难。

#### 解决方案：残差连接

$$
\text{Output} = \text{Sublayer}(x) + x
$$

为子层（Attention或FFN）的输出加上输入本身。

```
    x ──────────────┐
    │               │
    ↓               │
Sublayer            │
    │               │
    ↓               │
    + ←─────────────┘
    │
    ↓
  Output
```

#### 优势

1. **梯度直通**: 梯度可以直接流回,缓解梯度消失
2. **恒等映射**: 最坏情况下,网络可以学习恒等变换
3. **特征复用**: 保留浅层特征

#### 数学视角

反向传播时:

$$
\frac{\partial L}{\partial x} = \frac{\partial L}{\partial \text{Output}} \left(\frac{\partial \text{Sublayer}}{\partial x} + I\right)
$$

即使 $\frac{\partial \text{Sublayer}}{\partial x} \approx 0$,梯度仍然可以通过恒等项 $I$ 流回。

### 3.7 Layer Normalization (层归一化)

#### 归一化的作用

- 稳定训练
- 加速收敛
- 缓解梯度消失/爆炸

#### Layer Norm vs Batch Norm

| 维度 | Batch Norm | Layer Norm |
|------|-----------|------------|
| **归一化维度** | 批次维度 | 特征维度 |
| **适用场景** | CNN (固定batch size) | NLP (变长序列) |
| **计算方式** | 跨样本,同一特征 | 同一样本,跨特征 |

#### Layer Norm公式

对于输入 $x \in \mathbb{R}^{d}$:

$$
\mu = \frac{1}{d}\sum_{i=1}^{d} x_i
$$

$$
\sigma^2 = \frac{1}{d}\sum_{i=1}^{d} (x_i - \mu)^2
$$

$$
\text{LayerNorm}(x) = \gamma \frac{x - \mu}{\sqrt{\sigma^2 + \epsilon}} + \beta
$$

其中:
- $\gamma, \beta$ 是可学习的缩放和平移参数
- $\epsilon$ 是小常数(如1e-5)，防止除零

#### Pre-LN vs Post-LN

**原始Transformer (Post-LN)**:
```
x → Sublayer → Add → LayerNorm → ...
```

**现代架构 (Pre-LN, GPT-2后)**:
```
x → LayerNorm → Sublayer → Add → ...
```

Pre-LN训练更稳定,尤其是深层网络。

#### 其他归一化方法

- **RMSNorm**: LLaMA使用,只做缩放不做平移
  $$\text{RMSNorm}(x) = \frac{x}{\sqrt{\frac{1}{d}\sum x_i^2 + \epsilon}} \cdot \gamma$$
- **DeepNorm**: 针对超深模型的归一化策略

In [ ]:
# Layer Normalization的实现
def layer_norm(x, gamma, beta, eps=1e-5):
    """
    Layer Normalization
    
    参数:
    - x: 输入 (n, d)
    - gamma: 缩放参数 (d,)
    - beta: 平移参数 (d,)
    - eps: 防止除零的小常数
    """
    # 计算均值和方差（沿特征维度）
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    
    # 归一化
    x_norm = (x - mean) / np.sqrt(var + eps)
    
    # 缩放和平移
    out = gamma * x_norm + beta
    
    return out

# 示例
seq_len = 5
d_model = 4

# 随机输入
x = np.random.randn(seq_len, d_model) * 10 + 5  # 故意使用较大的均值和方差

# 初始化gamma和beta
gamma = np.ones(d_model)
beta = np.zeros(d_model)

# 应用Layer Norm
x_normalized = layer_norm(x, gamma, beta)

print("=== Layer Normalization Example ===")
print(f"\nOriginal input:\n{x}")
print(f"\nOriginal input statistics:")
print(f"- Row means: {np.mean(x, axis=1)}")
print(f"- Row standard deviations: {np.std(x, axis=1)}")

print(f"\nAfter normalization:\n{x_normalized}")
print(f"\nAfter normalization statistics:")
print(f"- Row means: {np.mean(x_normalized, axis=1)}")
print(f"- Row standard deviations: {np.std(x_normalized, axis=1)}")

# 可视化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.boxplot(x.T, labels=[f'Word{i+1}' for i in range(seq_len)])
ax1.set_title('Distribution Before Normalization')
ax1.set_ylabel('Value')
ax1.grid(True, alpha=0.3)

ax2.boxplot(x_normalized.T, labels=[f'Word{i+1}' for i in range(seq_len)])
ax2.set_title('Distribution After Normalization (Mean≈0, Std≈1)')
ax2.set_ylabel('Value')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 3.8 Masking (掩码)

掩码用于控制Attention机制中哪些位置可以被关注。

#### 3.8.1 Padding Mask

**问题**: 批处理时,不同样本的序列长度不同,需要填充(padding)到相同长度。

```
样本1: "我 爱 中国" (长度3)
样本2: "深度 学习 很 有趣" (长度4)

填充后:
样本1: "我 爱 中国 [PAD]"
样本2: "深度 学习 很 有趣"
```

**解决**: Padding Mask确保模型不关注填充位置。

```python
mask = (x != PAD_ID)  # [1, 1, 1, 0]
scores = scores + (1 - mask) * -1e9  # 填充位置得分→-∞
```

#### 3.8.2 Causal Mask (因果掩码)

**用途**: 自回归生成（如GPT）

**要求**: 生成第 $i$ 个词时,只能看到前 $i-1$ 个词,不能看到未来的词。

```
输入: "我 爱 中国"

Attention mask:
       我  爱  中  国
  我 [ 1   0   0   0 ]  ← "我"只能看自己
  爱 [ 1   1   0   0 ]  ← "爱"能看"我"和自己
  中 [ 1   1   1   0 ]  ← "中"能看"我爱"和自己
  国 [ 1   1   1   1 ]  ← "国"能看所有前文
```

**实现**: 使用下三角矩阵

```python
causal_mask = np.tril(np.ones((seq_len, seq_len)))
```

#### 3.8.3 Cross-Attention Mask

在Encoder-Decoder架构中,Decoder可以关注Encoder的所有位置,但需要Padding Mask。

#### 掩码类型总结

| 掩码类型 | 用途 | 应用场景 |
|---------|------|----------|
| **Padding Mask** | 忽略填充位置 | 所有Transformer |
| **Causal Mask** | 防止看到未来 | GPT等自回归模型 |
| **Custom Mask** | 特定任务需求 | 如结构化预测 |

In [ ]:
# 掩码的可视化
seq_len = 6

# 1. Causal Mask（因果掩码）
causal_mask = np.tril(np.ones((seq_len, seq_len)))

# 2. Padding Mask（假设最后两个位置是padding）
padding_mask = np.ones((seq_len, seq_len))
padding_mask[:, -2:] = 0  # 最后两列设为0
padding_mask[-2:, :] = 0  # 最后两行设为0

# 3. 组合掩码
combined_mask = causal_mask * padding_mask

# 可视化
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

masks = [causal_mask, padding_mask, combined_mask]
titles = ['Causal Mask\n(Autoregressive)', 'Padding Mask\n(Ignore Padding)', 'Combined Mask\n(Combined)']

for ax, mask, title in zip(axes, masks, titles):
    im = ax.imshow(mask, cmap='RdYlGn', vmin=0, vmax=1)
    ax.set_xticks(range(seq_len))
    ax.set_yticks(range(seq_len))
    ax.set_xticklabels([f'Pos{i}' for i in range(seq_len)], rotation=45)
    ax.set_yticklabels([f'Pos{i}' for i in range(seq_len)])
    ax.set_title(title)
    ax.set_xlabel('Key (Attended position)')
    ax.set_ylabel('Query (Current position)')
    
    # 添加数值
    for i in range(seq_len):
        for j in range(seq_len):
            val = int(mask[i, j])
            color = 'white' if val == 1 else 'black'
            ax.text(j, i, str(val), ha="center", va="center", 
                   color=color, fontsize=10, weight='bold')
    
    plt.colorbar(im, ax=ax, fraction=0.046)

plt.suptitle('Different Types of Attention Masks (1=Visible, 0=Invisible)', fontsize=14)
plt.tight_layout()
plt.show()

print("Mask descriptions:")
print("- Causal Mask: Can only see current position and earlier positions")
print("- Padding Mask: Positions 4 and 5 are padding, all positions should not attend to them")
print("- Combined Mask: Combination of both mask types")

## 4. Mixture of Experts (MoE)

MoE是一种扩展Transformer容量的技术,在不成比例增加计算量的情况下增加参数。

### 4.1 核心思想

将Feed-Forward Network替换为**多个专家网络**,每个token只激活部分专家。

```
传统FFN:
输入 → FFN → 输出

MoE:
输入 → Router → 选择Top-K专家 → 加权组合 → 输出
       ↓
    [ Expert 1 ]
    [ Expert 2 ]
    [ Expert 3 ]
    [   ...    ]
    [ Expert N ]
```

### 4.2 数学表达

对于输入 $x$:

1. **Router计算每个专家的权重**:
   $$G(x) = \text{Softmax}(x \cdot W_g)$$

2. **选择Top-K专家** (如K=2):
   $$\text{TopK}(G(x)) \rightarrow \{e_1, e_2\}$$

3. **加权组合专家输出**:
   $$\text{MoE}(x) = \sum_{i \in \text{TopK}} G_i(x) \cdot E_i(x)$$

其中 $E_i(x)$ 是第 $i$ 个专家的输出。

### 4.3 优势

✅ **参数效率**: 1.6T参数的MoE模型计算量≈100B稠密模型  
✅ **专业化**: 不同专家学习不同模式/领域  
✅ **扩展性**: 可以轻松增加专家数量

### 4.4 挑战

❌ **负载均衡**: 某些专家可能被过度使用  
❌ **训练不稳定**: Router需要特殊的正则化  
❌ **通信开销**: 分布式训练时专家间的通信

### 4.5 解决方案

1. **负载均衡损失**:
   $$L_{\text{balance}} = \alpha \cdot \text{CV}(\text{expert\_load})$$
   鼓励均匀分配token到各专家

2. **Expert Capacity**:
   限制每个专家处理的token数量上限

3. **Noise注入**:
   在Router中加入噪声增加探索

### 4.6 代表模型

| 模型 | 专家数 | Top-K | 总参数 | 激活参数 |
|------|-------|-------|--------|----------|
| **Switch Transformer** | 128-2048 | 1 | 1.6T | ~10B |
| **GLaM** | 64 | 2 | 1.2T | ~100B |
| **Mixtral 8x7B** | 8 | 2 | 47B | ~13B |
| **DeepSeek-V2** | 160 | 6 | 236B | ~21B |
| **Qwen2-57B-A14B** | ~64 | variable | 57B | ~14B |

### 4.7 MoE的未来

- **细粒度MoE**: 专家粒度更小（如multi-head层面）
- **动态MoE**: 根据任务动态调整专家数量
- **多模态MoE**: 不同专家处理不同模态（文本/图像/音频）

In [ ]:
# MoE概念可视化
def visualize_moe():
    """
    可视化MoE的专家选择过程
    """
    num_tokens = 8
    num_experts = 8
    top_k = 2
    
    # 模拟router为每个token计算的专家权重
    np.random.seed(42)
    router_logits = np.random.randn(num_tokens, num_experts)
    router_probs = np.exp(router_logits) / np.sum(np.exp(router_logits), axis=1, keepdims=True)
    
    # 选择Top-K专家
    topk_indices = np.argsort(router_probs, axis=1)[:, -top_k:]
    
    # 创建选择矩阵
    selection_matrix = np.zeros((num_tokens, num_experts))
    for i in range(num_tokens):
        for j in topk_indices[i]:
            selection_matrix[i, j] = router_probs[i, j]
    
    # 可视化
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    # 左图：完整的router概率
    im1 = ax1.imshow(router_probs, cmap='YlOrRd', aspect='auto')
    ax1.set_xlabel('Expert ID')
    ax1.set_ylabel('Token ID')
    ax1.set_title('Router Weights for Each Token')
    ax1.set_xticks(range(num_experts))
    ax1.set_yticks(range(num_tokens))
    plt.colorbar(im1, ax=ax1, label='Weight')
    
    # 右图：Top-K选择后
    im2 = ax2.imshow(selection_matrix, cmap='YlOrRd', aspect='auto')
    ax2.set_xlabel('Expert ID')
    ax2.set_ylabel('Token ID')
    ax2.set_title(f'Top-{top_k} Expert Selection (Black=Not Selected)')
    ax2.set_xticks(range(num_experts))
    ax2.set_yticks(range(num_tokens))
    plt.colorbar(im2, ax=ax2, label='Weight')
    
    plt.tight_layout()
    plt.show()
    
    # 统计专家负载
    expert_load = (selection_matrix > 0).sum(axis=0)
    
    # 绘制负载分布
    plt.figure(figsize=(10, 4))
    plt.bar(range(num_experts), expert_load, color='steelblue')
    plt.xlabel('Expert ID')
    plt.ylabel('Number of Tokens Processed')
    plt.title('Expert Load Distribution (Ideal: Uniform Distribution)')
    plt.axhline(y=num_tokens*top_k/num_experts, color='r', linestyle='--', 
               label=f'Ideal Load = {num_tokens*top_k/num_experts:.1f}')
    plt.xticks(range(num_experts))
    plt.grid(True, alpha=0.3, axis='y')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    print(f"\nMoE Statistics:")
    print(f"- Total tokens: {num_tokens}")
    print(f"- Number of experts: {num_experts}")
    print(f"- Top-K: {top_k}")
    print(f"- Percentage of activated parameters per token: {top_k/num_experts*100:.1f}%")
    print(f"- Expert load: {expert_load}")

visualize_moe()

## 5. Transformer的优势与局限

### 5.1 优势 ✅

1. **并行化**
   - 不像RNN需要顺序处理
   - 可以充分利用GPU/TPU的并行能力
   - 训练速度快几个数量级

2. **长距离依赖**
   - Self-Attention直接建模任意两个位置的关系
   - 路径长度为O(1)，RNN为O(n)
   - 更容易捕捉长距离依赖

3. **可解释性**
   - Attention权重可视化
   - 能看到模型关注哪些信息

4. **通用性**
   - 适用于多种任务和模态
   - 成为统一的深度学习架构

5. **可扩展性**
   - Scaling Law: 性能随模型大小、数据量持续提升
   - 已扩展到万亿参数级别

### 5.2 局限 ❌

1. **计算复杂度**
   - Self-Attention: $O(n^2 \cdot d)$
   - 序列长度翻倍,计算量翻4倍
   - 限制了处理超长序列的能力

2. **内存占用**
   - 需要存储 $n \times n$ 的attention矩阵
   - 长序列时内存爆炸

3. **位置编码**
   - 外推性有限（对超长序列）
   - 仍在探索更好的编码方式

4. **数据需求**
   - 需要大量数据才能充分发挥性能
   - 小数据集上可能不如RNN/CNN

### 5.3 改进方向

#### 高效Attention

| 方法 | 复杂度 | 代表模型 |
|------|--------|----------|
| **Sparse Attention** | $O(n \sqrt{n})$ | Sparse Transformer |
| **Linear Attention** | $O(n)$ | Linformer, Performer |
| **Sliding Window** | $O(n \cdot w)$ | Longformer |
| **Flash Attention** | $O(n^2)$但IO优化 | GPT-4, LLaMA |

#### 架构创新

- **Hybrid Models**: 结合RNN/CNN和Transformer（如RWKV）
- **State Space Models**: Mamba, S4等线性复杂度模型
- **Retrieval-Augmented**: 结合外部知识库,处理超长文档

### 5.4 Transformer复杂度对比

| 架构 | 每层复杂度 | 顺序操作 | 最大路径长度 |
|------|-----------|---------|-------------|
| **Self-Attention** | $O(n^2 \cdot d)$ | $O(1)$ | $O(1)$ |
| **RNN** | $O(n \cdot d^2)$ | $O(n)$ | $O(n)$ |
| **CNN** | $O(k \cdot n \cdot d^2)$ | $O(1)$ | $O(\log_k n)$ |

其中:
- $n$: 序列长度
- $d$: 表示维度
- $k$: 卷积核大小

## 6. 总结与展望

### 6.1 核心知识回顾

✅ **Embedding**: 将离散token映射到连续向量空间  
✅ **Positional Encoding**: 注入位置信息  
✅ **Self-Attention**: 建模序列内部的依赖关系  
✅ **Multi-Head Attention**: 多视角关注不同模式  
✅ **Feed-Forward Network**: 特征变换  
✅ **Residual Connection**: 缓解梯度消失  
✅ **Layer Normalization**: 稳定训练  
✅ **Masking**: 控制信息流动  
✅ **MoE**: 高效扩展模型容量

### 6.2 Transformer的演化

```
2017: Transformer (Encoder-Decoder)
  ↓
2018: BERT (Encoder-only, 双向)
      GPT-1 (Decoder-only, 自回归)
  ↓
2019: GPT-2, RoBERTa, T5, ALBERT
  ↓
2020: GPT-3, ViT (视觉), DETR (检测)
  ↓
2021: Switch Transformer (MoE), CLIP (多模态)
  ↓
2022: PaLM, Flamingo, ChatGPT
  ↓
2023: GPT-4, LLaMA, Claude, PaLM-2
  ↓
2024: Mixtral, Gemini, Claude 3, Qwen2, DeepSeek-V2
  ↓
2025: ...
```

### 6.3 从Transformer到LLM

现代大语言模型的核心都是Transformer:

**GPT系列** (Decoder-only):
- 自回归语言建模
- 因果掩码
- 纯文本生成

**BERT系列** (Encoder-only):
- 掩码语言建模
- 双向编码
- 文本理解任务

**T5/BART** (Encoder-Decoder):
- 序列到序列
- 适合翻译、摘要

### 6.4 Transformer之外

虽然Transformer主导当前深度学习，但也有替代架构:

- **RWKV**: 线性复杂度,结合RNN和Transformer优势
- **Mamba/S4**: 状态空间模型，$O(n)$复杂度
- **RetNet**: 保留训练并行性和推理效率

### 6.5 延伸阅读

📚 **必读论文**:
- "Attention Is All You Need" - Vaswani et al., 2017
- "BERT: Pre-training of Deep Bidirectional Transformers" - Devlin et al., 2018
- "Language Models are Few-Shot Learners" (GPT-3) - Brown et al., 2020

🌐 **推荐资源**:
- [The Illustrated Transformer](http://jalammar.github.io/illustrated-transformer/)
- [The Annotated Transformer](http://nlp.seas.harvard.edu/annotated-transformer/)
- Hugging Face Transformers库文档

---

## 课后思考

1. 为什么Self-Attention能够更好地捕捉长距离依赖，而RNN做不到？

2. Multi-Head Attention中，如果所有的头学习到了相同的模式，会发生什么？如何避免？

3. Transformer的 $O(n^2)$ 复杂度在什么场景下会成为瓶颈？有哪些解决方案？

4. MoE能否无限增加专家数量来提升性能？为什么？

5. 为什么现代LLM几乎都选择Decoder-only架构而不是Encoder-Decoder？

---

**下一课预告**: 第4课 - 主流大语言模型  
我们将了解基于Transformer构建的各种大语言模型（Qwen、GPT、LLaMA等），它们的架构差异、训练方法和应用场景！